# Conversation Analysis & Branch Detection - Interactive Notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chatroutes/chatroutes-autobranch/blob/master/notebooks/conversation_analysis_colab.ipynb)

Welcome! This notebook demonstrates **advanced conversation analysis** with intelligent branch detection.

## What You'll Learn

1. **Hybrid Detection** - Explicit + Semantic branch detection (no LLM needed!)
2. **Conversation Analysis** - Track topic shifts, decision points, Q→A transitions
3. **Flexible LLM Integration** - Use ANY LLM (OpenAI, Anthropic, Ollama, Groq, etc.)
4. **Intelligent Routing** - Automatic decision: when to use LLM vs hybrid
5. **Real-World Examples** - Article planning, conversation mapping

## Key Features

- ⚡ **Fast by default** - Hybrid detection is instant and free
- 🎯 **Opt-in LLM** - Only use LLM when you explicitly enable it
- 🧠 **Smart routing** - Heuristics decide when LLM adds value
- 🔌 **Pluggable** - Works with any LLM provider

---

Let's get started! 🚀

## Step 1: Installation

Install the latest version (takes ~30 seconds):

In [ ]:
!pip install --no-cache-dir --upgrade -q chatroutes-autobranch

# Show installed version
import chatroutes_autobranch
print(f"✅ Installation complete! Version: {chatroutes_autobranch.__version__}")

## Step 2: Import Components

In [ ]:
from chatroutes_autobranch.branch_detection import (
    BranchExtractor,
    ConversationFlowAnalyzer,
    ConversationTurn,
    LLMBranchParser,
    AutoRouter,
    IntelligentRouter,
)
from chatroutes_autobranch.core.embeddings import DummyEmbeddingProvider

print("✅ Imports successful!")
print("\n📦 Available Components:")
print("  • BranchExtractor - Detects explicit patterns")
print("  • ConversationFlowAnalyzer - Hybrid detection (explicit + semantic)")
print("  • LLMBranchParser - Optional LLM enhancement")
print("  • AutoRouter - Intelligent LLM routing")

## Example 1: Basic Pattern Detection (Instant, No LLM)

Start with the fastest method - pure pattern matching:

In [ ]:
# Sample conversation with clear decisions
text = """
For your project, consider:

BRANCH: Tech Stack Decision
OPTIONS:
1. Python + Flask - Simple, great for beginners
2. Node.js + Express - JavaScript everywhere
3. Go + Gin - High performance

Database: Use PostgreSQL or MySQL.

If you need caching then add Redis else skip it.
"""

# Extract patterns (instant!)
extractor = BranchExtractor()
branches = extractor.extract(text)

print(f"⚡ INSTANT DETECTION (No LLM)")
print(f"Found {len(branches)} branch points:\n")

for i, branch in enumerate(branches, 1):
    print(f"{i}. {branch.type.upper()}")
    print(f"   Options: {branch.get_option_labels()}")
    print()

stats = extractor.get_statistics(branches)
print(f"\n📊 Total combinations: {stats['max_leaves']}")
print(f"💡 Insight: 3 stacks × 2 databases × 2 cache options = {3*2*2} paths")

## Example 2: Conversation Flow Analysis (Hybrid)

Analyze a natural conversation - finds topic shifts, decision points, Q→A transitions:

In [ ]:
# Real conversation example
conversation = [
    ConversationTurn(
        id="1",
        speaker="user",
        content="I'm wondering whether to focus on philosophy or practical advice for my article."
    ),
    ConversationTurn(
        id="2",
        speaker="user",
        content="Actually, I want to do both. Maybe a multi-article series."
    ),
    ConversationTurn(
        id="3",
        speaker="user",
        content="""BRANCH: Article Structure
OPTIONS:
1. Philosophical - Deep dive into meaning and value
2. Practical - Actionable advice for professionals  
3. Historical - Pattern analysis from past transitions
4. Comprehensive - Multi-article covering all angles"""
    ),
    ConversationTurn(
        id="4",
        speaker="user",
        content="I'll go comprehensive. Let me plan the structure now."
    ),
]

# Set up hybrid analyzer (explicit + semantic)
analyzer = ConversationFlowAnalyzer(
    embedding_provider=DummyEmbeddingProvider(dimension=384, seed=42),
    topic_shift_threshold=0.6,
    enable_explicit=True,
    enable_semantic=True,
)

# Analyze (still instant!)
results = analyzer.analyze(conversation)

print("🔍 HYBRID ANALYSIS RESULTS\n")
print("=" * 50)

# Explicit branches
print(f"\n✓ Explicit Branches: {len(results['explicit_branches'])}")
for branch in results['explicit_branches']:
    print(f"  • Turn {branch.meta.get('turn_id')}: {branch.option_count} options")
    for opt in branch.options:
        print(f"    - {opt.label[:60]}")

# Semantic branches
print(f"\n✓ Semantic Branches: {len(results['semantic_branches'])}")
for branch in results['semantic_branches']:
    print(f"  • Turn {branch.turn_id}: {branch.branch_type}")
    print(f"    {branch.description}")
    print(f"    Confidence: {branch.confidence:.2f}")

total = len(results['explicit_branches']) + len(results['semantic_branches'])
print(f"\n📊 Total: {total} branches detected")
print("\n💡 Key insight: Hybrid found both explicit markers AND implicit patterns!")

## Example 3: Flexible LLM Integration

Choose YOUR LLM provider! Works with OpenAI, Anthropic, Ollama, Groq, or any custom LLM.

### Option A: OpenAI (GPT-3.5, GPT-4)

In [ ]:
# Install OpenAI (uncomment if you want to use)
# !pip install -q openai

def setup_openai_llm(api_key: str):
    """Set up OpenAI as LLM provider."""
    import openai
    
    def llm(prompt: str) -> str:
        client = openai.OpenAI(api_key=api_key)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # or "gpt-4"
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
        )
        return response.choices[0].message.content
    
    return llm

# Example usage (replace with your key)
# my_llm = setup_openai_llm("your-api-key-here")
# parser = LLMBranchParser(llm=my_llm)
# branches = parser.parse(conversation_text)

print("✅ OpenAI setup function defined")
print("   Usage: my_llm = setup_openai_llm('your-api-key')")

### Option B: Anthropic (Claude)

In [ ]:
# Install Anthropic (uncomment if you want to use)
# !pip install -q anthropic

def setup_anthropic_llm(api_key: str):
    """Set up Anthropic Claude as LLM provider."""
    import anthropic
    
    def llm(prompt: str) -> str:
        client = anthropic.Anthropic(api_key=api_key)
        response = client.messages.create(
            model="claude-3-haiku-20240307",  # or claude-3-sonnet, opus
            max_tokens=2000,
            messages=[{"role": "user", "content": prompt}],
        )
        return response.content[0].text
    
    return llm

# Example usage
# my_llm = setup_anthropic_llm("your-api-key-here")
# parser = LLMBranchParser(llm=my_llm)

print("✅ Anthropic setup function defined")
print("   Usage: my_llm = setup_anthropic_llm('your-api-key')")

### Option C: Groq (Fastest Cloud LLM)

In [ ]:
# Install Groq (uncomment if you want to use)
# !pip install -q groq

def setup_groq_llm(api_key: str):
    """Set up Groq (fast Llama 3) as LLM provider."""
    from groq import Groq
    
    def llm(prompt: str) -> str:
        client = Groq(api_key=api_key)
        response = client.chat.completions.create(
            model="llama3-70b-8192",  # or llama3-8b-8192
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
        )
        return response.choices[0].message.content
    
    return llm

# Example usage
# my_llm = setup_groq_llm("your-api-key-here")
# parser = LLMBranchParser(llm=my_llm)

print("✅ Groq setup function defined")
print("   Usage: my_llm = setup_groq_llm('your-api-key')")
print("   Note: Groq is VERY fast (~1s) and cheap!")

### Option D: Ollama (Local, Free)

If running locally (not in Colab), you can use Ollama:

In [ ]:
# Only works if Ollama is installed locally
# !pip install -q ollama

def setup_ollama_llm():
    """Set up Ollama (local Llama 3) as LLM provider."""
    import ollama
    
    def llm(prompt: str) -> str:
        response = ollama.chat(
            model='llama3',
            messages=[{'role': 'user', 'content': prompt}],
            options={'temperature': 0.1}
        )
        return response['message']['content']
    
    return llm

# Example usage (only works locally)
# my_llm = setup_ollama_llm()
# parser = LLMBranchParser(llm=my_llm)

print("✅ Ollama setup function defined")
print("   Note: Only works if Ollama installed locally")
print("   Install: https://ollama.ai")

### Option E: Custom LLM (Bring Your Own!)

You can use ANY LLM - just provide a function that takes a prompt and returns text:

In [ ]:
def my_custom_llm(prompt: str) -> str:
    """
    Your custom LLM function.
    
    Requirements:
    - Takes a string prompt as input
    - Returns a string response
    - That's it!
    """
    # Call your LLM API here
    # response = your_llm_api.call(prompt)
    # return response
    
    # Placeholder for demo
    return '{"branch_points": []}'

# Use it
parser = LLMBranchParser(llm=my_custom_llm)

print("✅ Custom LLM setup")
print("   The parser works with ANY function that:")
print("     1. Takes: string prompt")
print("     2. Returns: string response")

## Example 4: Intelligent LLM Routing (Auto-Decide)

Let the system decide when to use LLM vs hybrid:

In [ ]:
# Test conversations with different characteristics
test_conversations = {
    "Clear & Explicit": [
        ConversationTurn(
            id="1",
            speaker="user",
            content="""BRANCH: Choose framework
1. Flask
2. FastAPI"""
        ),
    ],
    
    "Ambiguous & Implicit": [
        ConversationTurn(
            id="1",
            speaker="user",
            content="I'm torn between philosophy and practical advice. On one hand, deep meaning matters. On the other hand, readers need actionable steps. Perhaps it depends on the audience..."
        ),
    ],
}

# Set up analyzer and router
analyzer = ConversationFlowAnalyzer(
    embedding_provider=DummyEmbeddingProvider(dimension=384, seed=42),
    enable_explicit=True,
    enable_semantic=True,
)

# Note: LLM is DISABLED by default
router = AutoRouter(
    analyzer=analyzer,
    # llm_parser=parser,  # Would enable if we had LLM configured
    enable_llm_routing=False,  # DEFAULT: LLM disabled
)

print("🧠 INTELLIGENT ROUTING DEMO\n")
print("=" * 60)

for name, conv in test_conversations.items():
    print(f"\n📝 Testing: {name}")
    print("-" * 60)
    
    results = router.analyze(conv, verbose=True)
    
    total = len(results['all_branches'])
    print(f"\n   Result: {total} branches detected")
    print()

print("\n💡 Key Insights:")
print("   • Default: LLM is DISABLED (fast, free)")
print("   • Opt-in: Set enable_llm_routing=True")
print("   • Smart: Router decides when LLM adds value")
print("   • Hybrid catches most patterns anyway!")

## 🎮 Interactive: Analyze Your Own Conversation!

Edit the conversation below and see what branches are detected:

In [ ]:
# ✏️ EDIT THIS - Add your own conversation!
your_conversation = [
    ConversationTurn(
        id="1",
        speaker="user",
        content="I'm planning a new project. Should I use Python or JavaScript?"
    ),
    ConversationTurn(
        id="2",
        speaker="user",
        content="Actually, maybe I should consider Go for performance. Or Rust?"
    ),
    ConversationTurn(
        id="3",
        speaker="user",
        content="""Let me be explicit:
        
BRANCH: Language Choice
OPTIONS:
1. Python - Easy to learn, great libraries
2. JavaScript - Web-focused, huge ecosystem
3. Go - Fast, great for services
4. Rust - Maximum performance and safety"""
    ),
]

# Analyze
analyzer = ConversationFlowAnalyzer(
    embedding_provider=DummyEmbeddingProvider(dimension=384, seed=42),
    enable_explicit=True,
    enable_semantic=True,
)

results = analyzer.analyze(your_conversation)

print("YOUR CONVERSATION ANALYSIS")
print("=" * 60)
print(f"\nExplicit branches: {len(results['explicit_branches'])}")
print(f"Semantic branches: {len(results['semantic_branches'])}")

for i, branch in enumerate(results['explicit_branches'], 1):
    print(f"\n{i}. Explicit: {branch.option_count} options")
    for opt in branch.options:
        print(f"   • {opt.label[:60]}")

for i, branch in enumerate(results['semantic_branches'], 1):
    print(f"\n{i}. Semantic: {branch.branch_type}")
    print(f"   {branch.description}")

print(f"\n📊 Total: {len(results['explicit_branches']) + len(results['semantic_branches'])} branches")

## 🎯 Real-World Example: Article Planning

Analyze a conversation about planning an article series:

In [ ]:
article_conversation = [
    ConversationTurn(
        id="1",
        speaker="user",
        content="I want to write about AI impact on human work."
    ),
    ConversationTurn(
        id="2",
        speaker="user",
        content="Should I focus on philosophy or practical advice?"
    ),
    ConversationTurn(
        id="3",
        speaker="user",
        content="""I think I'll do a multi-article series:
        
BRANCH: Series Structure
OPTIONS:
1. Article 1: Philosophy (creativity, meaning, value)
2. Article 2: Professions (accountants, lawyers, doctors)
3. Article 3: Companies (traditional vs AI-native)
4. Article 4: Education (what to study, skills)"""
    ),
    ConversationTurn(
        id="4",
        speaker="user",
        content="I'll start with Article 1. Need historical references and credible sources."
    ),
]

analyzer = ConversationFlowAnalyzer(
    embedding_provider=DummyEmbeddingProvider(dimension=384, seed=42),
    enable_explicit=True,
    enable_semantic=True,
)

results = analyzer.analyze(article_conversation)

print("📚 ARTICLE PLANNING ANALYSIS")
print("=" * 60)

print("\n🔍 Conversation Flow:")
print("  Turn 1: Initial idea (AI & work)")
print("  Turn 2: Question (philosophy vs practical)")
print("  Turn 3: Decision (multi-article series) ← BRANCH")
print("  Turn 4: Action (start with Article 1) ← COMMITMENT")

print(f"\n📊 Branches Detected:")
print(f"  Explicit: {len(results['explicit_branches'])}")
print(f"  Semantic: {len(results['semantic_branches'])}")

if results['explicit_branches']:
    branch = results['explicit_branches'][0]
    print(f"\n📝 Article Series Structure ({branch.option_count} articles):")
    for opt in branch.options:
        print(f"  • {opt.label}")

for branch in results['semantic_branches']:
    print(f"\n🧠 Semantic: {branch.branch_type}")
    print(f"   Turn {branch.turn_id}: {branch.description}")

print("\n💡 Insight: The conversation evolved from question → decision → action!")

## 📊 Summary & Best Practices

### What You Learned

✅ **Pattern Detection** - Instant, deterministic branch extraction  
✅ **Hybrid Analysis** - Explicit + Semantic without LLM  
✅ **Flexible LLM Integration** - Works with ANY provider  
✅ **Intelligent Routing** - Auto-decides when to use LLM  
✅ **Real-World Applications** - Article planning, conversation analysis  

### Performance Comparison

| Method | Speed | Cost | Detection Quality |
|--------|-------|------|------------------|
| Pattern Only | <0.1s | $0 | Good for explicit |
| Hybrid (default) | <0.1s | $0 | Excellent for most cases |
| + LLM (opt-in) | 1-5s | ~$0.0001 | Best for complex/ambiguous |

### Best Practices

1. **Start with Hybrid** - It's instant, free, and catches most patterns
2. **Add explicit markers** - Use `BRANCH:` and `OPTIONS:` for critical decisions
3. **Opt-in to LLM** - Only enable when you need deeper analysis
4. **Choose your LLM** - Works with OpenAI, Anthropic, Ollama, Groq, or custom
5. **Trust the heuristics** - Intelligent routing makes good decisions

### LLM Provider Comparison

| Provider | Speed | Cost | Setup |
|----------|-------|------|-------|
| **Hybrid (No LLM)** | ⚡ Instant | 💰 Free | ✅ Works now |
| **Groq** | ⚡ Fast (~1s) | 💰 Cheap | 🔑 API key |
| **OpenAI** | 🐌 2-3s | 💰💰 Moderate | 🔑 API key |
| **Anthropic** | 🐌 2-3s | 💰💰 Moderate | 🔑 API key |
| **Ollama** | ⚡ 2-5s | 💰 Free | 💻 Local only |

**Recommendation:** Start with hybrid, add Groq if you need LLM (fastest & cheapest).

## 🔗 Next Steps

### Explore More Features

This notebook focused on branch detection. The library also includes:

- **Beam Search** - Select top-K candidates
- **Novelty Filtering** - Remove similar options
- **Entropy Stopping** - Know when to stop branching
- **Budget Management** - Control costs and tokens

### Resources

- 📖 [GitHub Repository](https://github.com/chatroutes/chatroutes-autobranch)
- 📦 [PyPI Package](https://pypi.org/project/chatroutes-autobranch/)
- 📚 [Full Documentation](https://github.com/chatroutes/chatroutes-autobranch/blob/master/README.md)
- 💡 [More Examples](https://github.com/chatroutes/chatroutes-autobranch/tree/master/examples)

### Try It Locally

```bash
pip install chatroutes-autobranch

# Run examples
python examples/analyze_conversation_hybrid.py
python examples/intelligent_routing_demo.py
python examples/llama3_branch_detection.py
```

---

**Happy analyzing!** 🎯🚀